In [19]:
import pandas as pd
from numpy import asarray
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
import pickle

### Linear regression

répartition des données

In [20]:
df = pd.read_csv('house_data_cleaned.csv')
y = df['price']
X = df[[
#     'id',
    # 'date',
    'bedrooms',
    'bathrooms',
    'sqft_living',
    'sqft_lot',
    'floors',
    'waterfront',
    'view',
    'condition',
    'grade',
    'sqft_above',    
    # 'sqft_basement',
    'yr_built',
    'yr_renovated',
    'zipcode', #
    'lat',
    'long',
#     'sqft_living15',
#     'sqft_lot15',
    # 'yr_best',
    # 'yr_sold',
    'yr_gap',
    # 'comparison',
    ]]

### Preparation du pipeline

In [21]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, random_state=42
    )

numeric_features = [
    # 'date',
    'bedrooms',
    'bathrooms',
    'sqft_living',
    'sqft_lot',
    'yr_built',
    'yr_renovated',
    'floors',
    'sqft_above',
    # 'sqft_basement',
    'lat',
    'long',
    # 'yr_best',
    # 'yr_sold',
    'yr_gap',
    # 'comparison'
    ]
numeric_transformer_minmax = Pipeline([
    ('minmax', MinMaxScaler())
    ])

numeric_transformer_std = Pipeline([
    ('standard', StandardScaler())
    ])

numeric_transformer_rbst = Pipeline([
    ('standard', RobustScaler()),
    ])

In [22]:
categorial_features = [
    'view',
    'condition',
    'grade',
    'zipcode',
    ]

categorial_transformer = OneHotEncoder(sparse_output=True)

In [23]:
preprocessor_minmax = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_minmax, numeric_features),
        ('cat', categorial_transformer, categorial_features)
    ],
    remainder='passthrough'
)

preprocessor_std = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_std, numeric_features),
        ('cat', categorial_transformer, categorial_features)
    ],
    remainder='passthrough'
)

preprocessor_rbst = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_rbst, numeric_features),
        ('cat', categorial_transformer, categorial_features)
    ],
    remainder='passthrough'
)

Estimateur

In [24]:
lnr = LinearRegression()

In [29]:
pipe = Pipeline([
    ('prep', preprocessor_std),
    ('lnr', lnr)
])

trained_pipe = pipe.fit(X_train, y_train)
trained_pipe.predict(X_test)
trained_pipe.score(X_test, y_test)

0.8385820435466707

historique des scores

In [30]:
scores = {
    'minmax_lnr' : 0.8387817757733049,
    'std_lnr' : 0.8387821140745632,
    'rbst_lnr' : 0.8385820435466707,
}

bestscore = max(scores.values())
bestscore = [k for (k, val) in scores.items() if val == bestscore]
print(bestscore[0])

minmax_std


essai du modele entrainé

In [27]:
import random as rd
randomint = rd.randint(0,len(X_test)+1)
predict = round(trained_pipe.predict(X_test.iloc[[randomint]])[0])
real = round(y.iloc[[randomint]].values[0])
difference = round(abs(real - predict))
accuracy = str(round((((real-difference) * 100) / real), 2)) + "%"
print(f"Valeur prédite : {predict} \n Valeur réelle : {real} \n Différence : {difference}, \n Précision : {accuracy}")

Valeur prédite : 654800 
 Valeur réelle : 352000 
 Différence : 302800, 
 Précision : 13.98%
